## Data
- Lets have a look at our dataset, preprocess it and save the preprocessed version

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.5 MB/s eta 0:00:00


In [2]:
import re
import pandas as pd

from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

In [3]:
# dataset comes from here: https://github.com/theochem/B3DB/blob/main/README.md

df = pd.read_csv("https://staicentreprod001.blob.core.windows.net/share/mlprague23/B3DB_classification.tsv", sep="\t")
df.head()

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN
1,2,moxalactam,7-[[2-carboxy-2-(4-hydroxyphenyl)acetyl]amino]...,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,3889.0,-2.52,BBB-,InChI=1S/C20H20N6O9S/c1-25-19(22-23-24-25)36-8...,NaN,R25|,A,NaN
2,3,clioquinol,5-chloro-7-iodoquinolin-8-ol,Oc1c(I)cc(Cl)c2cccnc12,2788.0,-2.40,BBB-,InChI=1S/C9H5ClINO/c10-6-4-7(11)9(13)8-5(6)2-1...,NaN,R18|R26|R27|,A,NaN
3,4,bbcpd11 (cimetidine analog) (y-g13),2-[2-[(3-bromopyridin-2-yl)methylsulfanyl]ethy...,CCNC(=NCCSCc1ncccc1Br)NC#N,14022517.0,-2.15,BBB-,InChI=1S/C12H16BrN5S/c1-2-15-12(18-9-14)17-6-7...,NaN,R2|R2|R8|R40|R2|R2|R2|R2|R18|R21|R25|R25|R26|R...,A,NaN
4,5,schembl614298,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-7-hydro...",CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,18595497.0,-2.15,BBB-,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-4-12(25)2...,NaN,R25|,A,NaN


In [5]:
df.to_csv("B3DB_classification.tsv", sep="\t", index=False)

In [4]:
df.shape

(7807, 12)

In [6]:
# for readability
df = df.rename(columns={"BBB+/BBB-": "label", "compound_name": "name"})
df.loc[df.label == "BBB+", "label"] = 1
df.loc[df.label == "BBB-", "label"] = 0

df.head(20)

,NO.,name,IUPAC_name,SMILES,CID,logBB,label,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,0,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN
1,2,moxalactam,7-[[2-carboxy-2-(4-hydroxyphenyl)acetyl]amino]...,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,3889.0,-2.52,0,InChI=1S/C20H20N6O9S/c1-25-19(22-23-24-25)36-8...,NaN,R25|,A,NaN
2,3,clioquinol,5-chloro-7-iodoquinolin-8-ol,Oc1c(I)cc(Cl)c2cccnc12,2788.0,-2.40,0,InChI=1S/C9H5ClINO/c10-6-4-7(11)9(13)8-5(6)2-1...,NaN,R18|R26|R27|,A,NaN
3,4,bbcpd11 (cimetidine analog) (y-g13),2-[2-[(3-bromopyridin-2-yl)methylsulfanyl]ethy...,CCNC(=NCCSCc1ncccc1Br)NC#N,14022517.0,-2.15,0,InChI=1S/C12H16BrN5S/c1-2-15-12(18-9-14)17-6-7...,NaN,R2|R2|R8|R40|R2|R2|R2|R2|R18|R21|R25|R25|R26|R...,A,NaN
4,5,schembl614298,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-7-hydro...",CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,18595497.0,-2.15,0,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-4-12(25)2...,NaN,R25|,A,NaN
5,6,"uk-240,455","n-(6,7-dichloro-2,3-dioxo-1,4-dihydroquinoxali...",CS(=O)(=O)N(CCO)c1c(Cl)c(Cl)cc2[nH]c(=O)c(=O)[...,9842188.0,-2.15,0,"InChI=1S/C11H11Cl2N3O5S/c1-22(20,21)16(2-3-17)...",NaN,R18|R26|R27|,A,NaN
6,7,morphine-6-glucuronide,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-9-hydro...",CN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](OC2O[C@H]...,9847115.0,-2.09,0,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-5-13(31-2...,NaN,R25|,A,NaN
7,8,nitrofurantoin,1-[(e)-(5-nitrofuran-2-yl)methylideneamino]imi...,O=C1CN(/N=C/c2ccc([N+](=O)[O-])o2)C(=O)N1,5353830.0,-2.09,0,InChI=1S/C8H6N4O5/c13-6-4-11(8(14)10-6)9-3-5-1...,NaN,R2|R2|R25|R46|,A,NaN
8,9,"l-701,324",7-chloro-4-hydroxy-3-(3-phenoxyphenyl)-1h-quin...,O=c1[nH]c2cc(Cl)ccc2c(O)c1-c1cccc(Oc2ccccc2)c1,54682505.0,-2.03,0,InChI=1S/C21H14ClNO3/c22-14-9-10-17-18(12-14)2...,NaN,R18|R26|R27|,A,NaN
9,10,33419-42-0,"5-[(7,8-dihydroxy-2-methyl-4,4a,6,7,8,8a-hexah...",COc1cc(C2c3cc4c(cc3C(OC3OC5COC(C)OC5C(O)C3O)C3...,3310.0,-2.00,0,InChI=1S/C29H32O13/c1-11-36-9-20-27(40-11)24(3...,NaN,R2|R2|R8|R12|R21|R25|R35|R40|R47|,A,NaN


### Do the molecule names need to be cleaned?

In [7]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

In [8]:
mol_ids = tokenizer.encode('bbcpd11 (cimetidine analog) (y-g13)')
print('mol_ids:', mol_ids)

print('mol_tokens', tokenizer.convert_ids_to_tokens(mol_ids))

mol_ids: [2, 11212, 10020, 7240, 1009, 12, 20090, 4988, 27099, 5729, 13, 12, 67, 17, 8533, 1010, 13, 3]
mol_tokens ['[CLS]', 'bb', '##cp', '##d1', '##1', '(', 'cim', '##eti', '##dine', 'analog', ')', '(', 'y', '-', 'g1', '##3', ')', '[SEP]']


- Notice how the subword unit (suffix) starts with "##" to indicate that it is part of the previous string
- Also [CLS] and [SEP] tokens are automatically added

In [9]:
mol_ids = tokenizer.encode('morphine-6-glucuronide')
print('mol_ids:', mol_ids)
print('mol_tokens', tokenizer.convert_ids_to_tokens(mol_ids))

mol_ids: [2, 11553, 17, 26, 17, 29306, 3]
mol_tokens ['[CLS]', 'morphine', '-', '6', '-', 'glucuronide', '[SEP]']


- The model has *morphine* and *glucuronide* in its vocabulary (has matching input id for these words)
- But doesn't have *bbcpd11* or *cimetidine*

In [10]:
mol_ids = tokenizer.encode('33419-42-0')
print('mol_ids:', mol_ids)
print('mol_tokens', tokenizer.convert_ids_to_tokens(mol_ids))

mol_ids: [2, 26403, 4136, 17, 3858, 17, 20, 3]
mol_tokens ['[CLS]', '334', '##19', '-', '42', '-', '0', '[SEP]']


#### Regex to remove non alpha-numeric characters and convert to lowercase
- see it in action: https://regex101.com/

In [11]:
df["name"] = df["name"].apply(lambda x: re.sub("[^A-Za-z0-9]+", "", str(x)).lower())
df.head(20)

,NO.,name,IUPAC_name,SMILES,CID,logBB,label,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,0,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN
1,2,moxalactam,7-[[2-carboxy-2-(4-hydroxyphenyl)acetyl]amino]...,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,3889.0,-2.52,0,InChI=1S/C20H20N6O9S/c1-25-19(22-23-24-25)36-8...,NaN,R25|,A,NaN
2,3,clioquinol,5-chloro-7-iodoquinolin-8-ol,Oc1c(I)cc(Cl)c2cccnc12,2788.0,-2.40,0,InChI=1S/C9H5ClINO/c10-6-4-7(11)9(13)8-5(6)2-1...,NaN,R18|R26|R27|,A,NaN
3,4,bbcpd11cimetidineanalogyg13,2-[2-[(3-bromopyridin-2-yl)methylsulfanyl]ethy...,CCNC(=NCCSCc1ncccc1Br)NC#N,14022517.0,-2.15,0,InChI=1S/C12H16BrN5S/c1-2-15-12(18-9-14)17-6-7...,NaN,R2|R2|R8|R40|R2|R2|R2|R2|R18|R21|R25|R25|R26|R...,A,NaN
4,5,schembl614298,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-7-hydro...",CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,18595497.0,-2.15,0,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-4-12(25)2...,NaN,R25|,A,NaN
5,6,uk240455,"n-(6,7-dichloro-2,3-dioxo-1,4-dihydroquinoxali...",CS(=O)(=O)N(CCO)c1c(Cl)c(Cl)cc2[nH]c(=O)c(=O)[...,9842188.0,-2.15,0,"InChI=1S/C11H11Cl2N3O5S/c1-22(20,21)16(2-3-17)...",NaN,R18|R26|R27|,A,NaN
6,7,morphine6glucuronide,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-9-hydro...",CN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](OC2O[C@H]...,9847115.0,-2.09,0,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-5-13(31-2...,NaN,R25|,A,NaN
7,8,nitrofurantoin,1-[(e)-(5-nitrofuran-2-yl)methylideneamino]imi...,O=C1CN(/N=C/c2ccc([N+](=O)[O-])o2)C(=O)N1,5353830.0,-2.09,0,InChI=1S/C8H6N4O5/c13-6-4-11(8(14)10-6)9-3-5-1...,NaN,R2|R2|R25|R46|,A,NaN
8,9,l701324,7-chloro-4-hydroxy-3-(3-phenoxyphenyl)-1h-quin...,O=c1[nH]c2cc(Cl)ccc2c(O)c1-c1cccc(Oc2ccccc2)c1,54682505.0,-2.03,0,InChI=1S/C21H14ClNO3/c22-14-9-10-17-18(12-14)2...,NaN,R18|R26|R27|,A,NaN
9,10,33419420,"5-[(7,8-dihydroxy-2-methyl-4,4a,6,7,8,8a-hexah...",COc1cc(C2c3cc4c(cc3C(OC3OC5COC(C)OC5C(O)C3O)C3...,3310.0,-2.00,0,InChI=1S/C29H32O13/c1-11-36-9-20-27(40-11)24(3...,NaN,R2|R2|R8|R12|R21|R25|R35|R40|R47|,A,NaN


In [12]:
# replace molecules whose names are just numbers with nan
df["name"] = df["name"].apply(lambda x: re.sub("^[0-9]+", "nan", str(x)))
df.head(20)

,NO.,name,IUPAC_name,SMILES,CID,logBB,label,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,0,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN
1,2,moxalactam,7-[[2-carboxy-2-(4-hydroxyphenyl)acetyl]amino]...,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,3889.0,-2.52,0,InChI=1S/C20H20N6O9S/c1-25-19(22-23-24-25)36-8...,NaN,R25|,A,NaN
2,3,clioquinol,5-chloro-7-iodoquinolin-8-ol,Oc1c(I)cc(Cl)c2cccnc12,2788.0,-2.40,0,InChI=1S/C9H5ClINO/c10-6-4-7(11)9(13)8-5(6)2-1...,NaN,R18|R26|R27|,A,NaN
3,4,bbcpd11cimetidineanalogyg13,2-[2-[(3-bromopyridin-2-yl)methylsulfanyl]ethy...,CCNC(=NCCSCc1ncccc1Br)NC#N,14022517.0,-2.15,0,InChI=1S/C12H16BrN5S/c1-2-15-12(18-9-14)17-6-7...,NaN,R2|R2|R8|R40|R2|R2|R2|R2|R18|R21|R25|R25|R26|R...,A,NaN
4,5,schembl614298,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-7-hydro...",CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,18595497.0,-2.15,0,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-4-12(25)2...,NaN,R25|,A,NaN
5,6,uk240455,"n-(6,7-dichloro-2,3-dioxo-1,4-dihydroquinoxali...",CS(=O)(=O)N(CCO)c1c(Cl)c(Cl)cc2[nH]c(=O)c(=O)[...,9842188.0,-2.15,0,"InChI=1S/C11H11Cl2N3O5S/c1-22(20,21)16(2-3-17)...",NaN,R18|R26|R27|,A,NaN
6,7,morphine6glucuronide,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-9-hydro...",CN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](OC2O[C@H]...,9847115.0,-2.09,0,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-5-13(31-2...,NaN,R25|,A,NaN
7,8,nitrofurantoin,1-[(e)-(5-nitrofuran-2-yl)methylideneamino]imi...,O=C1CN(/N=C/c2ccc([N+](=O)[O-])o2)C(=O)N1,5353830.0,-2.09,0,InChI=1S/C8H6N4O5/c13-6-4-11(8(14)10-6)9-3-5-1...,NaN,R2|R2|R25|R46|,A,NaN
8,9,l701324,7-chloro-4-hydroxy-3-(3-phenoxyphenyl)-1h-quin...,O=c1[nH]c2cc(Cl)ccc2c(O)c1-c1cccc(Oc2ccccc2)c1,54682505.0,-2.03,0,InChI=1S/C21H14ClNO3/c22-14-9-10-17-18(12-14)2...,NaN,R18|R26|R27|,A,NaN
9,10,nan,"5-[(7,8-dihydroxy-2-methyl-4,4a,6,7,8,8a-hexah...",COc1cc(C2c3cc4c(cc3C(OC3OC5COC(C)OC5C(O)C3O)C3...,3310.0,-2.00,0,InChI=1S/C29H32O13/c1-11-36-9-20-27(40-11)24(3...,NaN,R2|R2|R8|R12|R21|R25|R35|R40|R47|,A,NaN


In [13]:
df[df["name"] == "nan"]

,NO.,name,IUPAC_name,SMILES,CID,logBB,label,Inchi,threshold,reference,group,comments
9,10,nan,"5-[(7,8-dihydroxy-2-methyl-4,4a,6,7,8,8a-hexah...",COc1cc(C2c3cc4c(cc3C(OC3OC5COC(C)OC5C(O)C3O)C3...,3310.0,-2.00,0,InChI=1S/C29H32O13/c1-11-36-9-20-27(40-11)24(3...,NaN,R2|R2|R8|R12|R21|R25|R35|R40|R47|,A,NaN
15,16,nan,"n-phenyl-4,5-dihydro-1h-imidazol-2-amine",c1ccc(NC2=NCCN2)cc1,137235.0,-1.89,0,InChI=1S/C9H11N3/c1-2-4-8(5-3-1)12-9-10-6-7-11...,NaN,R2|R2|R8|R25|R46|R47|R4|R40|,A,NaN
20,21,nan,NaN,CN1Cc2c(-c3noc(C(C)(O)O)n3)ncn2-c2cccc(Cl)c2C1=O,NaN,-1.82,0,"InChI=1S/C16H14ClN5O4/c1-16(24,25)15-19-13(20-...",NaN,R35|,A,NaN
21,22,nan,NaN,CN1Cc2c(-c3noc([C@](C)(O)CO)n3)ncn2-c2cccc(Cl)...,NaN,-1.82,0,"InChI=1S/C17H16ClN5O4/c1-17(26,7-24)16-20-14(2...",NaN,R48|,A,NaN
55,56,nan,"8-(3-oxocyclopentyl)-1,3-dipropyl-7h-purine-2,...",CCCn1c(=O)c2[nH]c(C3CCC(=O)C3)nc2n(CCC)c1=O,131584.0,-1.40,0,InChI=1S/C16H22N4O3/c1-3-7-19-14-12(15(22)20(8...,NaN,R2|R2|R4|R27|R40|R43|R47|,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7696,7697,nan,NaN,O=c1[nH]c2ccccc2n1CCCN1CCC(Nn2c(=O)[nH]c3cc(Cl...,NaN,NaN,1,InChI=1S/C22H25ClN6O2/c23-15-6-7-20-18(14-15)2...,NaN,R27|,C,NaN
7700,7701,nan,NaN,OC[C@@H]1O[C@@H](OC[C@@H]2O[C@@H](O[C@]3(CO)O[...,NaN,NaN,0,InChI=1S/C18H32O16/c19-1-5-8(22)11(25)13(27)16...,NaN,R27|,C,NaN
7701,7702,nan,NaN,OC[C@@H]1O[C@@H](O[C@@H]2[C@H](CO)O[C@@](O)(CO...,NaN,NaN,0,InChI=1S/C12H22O11/c13-1-4-6(16)7(17)8(18)11(2...,NaN,R27|,C,NaN
7705,7706,nan,NaN,ON=Cc1cc[n+](COC[n+]2ccc(/C=N/O)cc2)cc1,NaN,NaN,0,InChI=1S/C14H14N4O3/c19-15-9-13-1-5-17(6-2-13)...,NaN,R27|,C,NaN


In [14]:
num_nan=sum(df["name"] == "nan")
print(f"number of molecules with nan name: {num_nan}")

number of molecules with nan name: 1279


In [15]:
print(f"df shape before nan molecule removal: {df.shape}")
df = df[df["name"] != "nan"]
print(f"df shape after nan molecule removal: {df.shape}")

df shape before nan molecule removal: (7807, 12)
df shape after nan molecule removal: (6528, 12)


In [16]:
df.head(20)

,NO.,name,IUPAC_name,SMILES,CID,logBB,label,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,0,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN
1,2,moxalactam,7-[[2-carboxy-2-(4-hydroxyphenyl)acetyl]amino]...,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,3889.0,-2.52,0,InChI=1S/C20H20N6O9S/c1-25-19(22-23-24-25)36-8...,NaN,R25|,A,NaN
2,3,clioquinol,5-chloro-7-iodoquinolin-8-ol,Oc1c(I)cc(Cl)c2cccnc12,2788.0,-2.40,0,InChI=1S/C9H5ClINO/c10-6-4-7(11)9(13)8-5(6)2-1...,NaN,R18|R26|R27|,A,NaN
3,4,bbcpd11cimetidineanalogyg13,2-[2-[(3-bromopyridin-2-yl)methylsulfanyl]ethy...,CCNC(=NCCSCc1ncccc1Br)NC#N,14022517.0,-2.15,0,InChI=1S/C12H16BrN5S/c1-2-15-12(18-9-14)17-6-7...,NaN,R2|R2|R8|R40|R2|R2|R2|R2|R18|R21|R25|R25|R26|R...,A,NaN
4,5,schembl614298,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-7-hydro...",CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,18595497.0,-2.15,0,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-4-12(25)2...,NaN,R25|,A,NaN
5,6,uk240455,"n-(6,7-dichloro-2,3-dioxo-1,4-dihydroquinoxali...",CS(=O)(=O)N(CCO)c1c(Cl)c(Cl)cc2[nH]c(=O)c(=O)[...,9842188.0,-2.15,0,"InChI=1S/C11H11Cl2N3O5S/c1-22(20,21)16(2-3-17)...",NaN,R18|R26|R27|,A,NaN
6,7,morphine6glucuronide,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-9-hydro...",CN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](OC2O[C@H]...,9847115.0,-2.09,0,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-5-13(31-2...,NaN,R25|,A,NaN
7,8,nitrofurantoin,1-[(e)-(5-nitrofuran-2-yl)methylideneamino]imi...,O=C1CN(/N=C/c2ccc([N+](=O)[O-])o2)C(=O)N1,5353830.0,-2.09,0,InChI=1S/C8H6N4O5/c13-6-4-11(8(14)10-6)9-3-5-1...,NaN,R2|R2|R25|R46|,A,NaN
8,9,l701324,7-chloro-4-hydroxy-3-(3-phenoxyphenyl)-1h-quin...,O=c1[nH]c2cc(Cl)ccc2c(O)c1-c1cccc(Oc2ccccc2)c1,54682505.0,-2.03,0,InChI=1S/C21H14ClNO3/c22-14-9-10-17-18(12-14)2...,NaN,R18|R26|R27|,A,NaN
10,11,icotidine,2-[4-(3-methoxypyridin-2-yl)butylamino]-5-[(6-...,COc1cccnc1CCCCNc1ncc(Cc2ccc(C)nc2)c(=O)[nH]1,72108.0,-2.00,0,InChI=1S/C21H25N5O2/c1-15-8-9-16(13-24-15)12-1...,NaN,R2|R2|R2|R2|R5|R8|R11|R12|R18|R21|R25|R26|R27|...,A,NaN


In [17]:
df = df.drop_duplicates(subset="name")
df.shape

(4832, 12)

In [28]:
# make train-test split of data using name column as X data and label as y data
X = df.name.values
y = df.label.values
X_train, X_test, y_train, y_test = train_test_split(X, y,  stratify=y, test_size=0.2, random_state=42)

In [29]:
df.name.values

array(['sulphasalazine', 'moxalactam', 'clioquinol', ..., 'ketotifen',
       'milrinone', 'amrinone'], dtype=object)

In [30]:
df_train = pd.DataFrame(data={"name": X_train, "label": y_train})

df_test = pd.DataFrame(data={"name": X_test, "label": y_test})

Mount your drive to colab so you can write the processed data there

In [31]:
# Do this only on colab
# from google.colab import drive

# drive.mount('/content/drive', force_remount=True)

In [32]:
# save preprocessed data to your drive 
df_train.to_csv('../data/B3DB_cleaned_name_TRAIN_nb.csv', index=False)
df_test.to_csv('../data/B3DB_cleaned_name_TEST_nb.csv', index=False)

Split and save dataset with SMILES separately

In [33]:
# repeat what was done above using name column but now for SMILES, and save the data :)
X = df["SMILES"].values
y = df.label.values
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [34]:
df_train = pd.DataFrame(data={"SMILES": X_train, "label": y_train})

df_test = pd.DataFrame(data={"SMILES": X_test, "label": y_test})

df_train.to_csv("../data/B3DB_cleaned_SMILES_TRAIN_nb.csv", index=False)
df_test.to_csv("../data/B3DB_cleaned_SMILES_TEST_nb.csv", index=False)